## Outline
1. Lists and Pattern Matching
2. Higher-order Functions
3. Abstract Syntax Tree


## Lists and Pattern Matching

Our inductive structure of choice for this problem is a linked list. The below code defines the structure.

In [ ]:
sealed trait NumList
case object MyNil extends NumList
case class MyCons(firstElement : Int, restOfList : NumList) extends NumList

Since it's an inductive structure, we can write a grammar for it:

$$
\begin{array}{rcl}
    \textbf{NumList} & \rightarrow & MyNil \\
                     &           | & MyCons(\textbf{Int}, \textbf{NumList}) \\
\end{array}
$$

### Exercise: Writing lists
Write out the following lists with our `NumList` class.

1. `[]` (The empty list)
2. `[1, 2, 3]`

In [ ]:
// BEGIN SOLUTION
val ans_1 = MyNil
val ans_2 = MyCons(1, MyCons(2, MyCons(3, MyNil)))
// END SOLUTION

### Exercise: Length of list
Implement a `myLength` function for our list type using [pattern matching](https://docs.scala-lang.org/tour/pattern-matching.html).

In [ ]:
// Is this function recursive?
// Is it tail recursive?

def myLength(list : NumList) : Int = {
    // BEGIN SOLUTION
    list match {
        case MyNil => 0
        case MyCons(firstElement, restOfList) => 1 + myLength(restOfList)
    }
    // END SOLUTION
}

In [ ]:
assert(myLength(MyNil) == 0)
assert(myLength(MyCons(1, MyCons(2, MyNil))) == 2)

### Exercise: Translate to built in lists
Rewrite `length` to use [Scala's list class](https://www.scala-lang.org/api/current/scala/collection/immutable/List.html), which is very similar to the one defined above. The table below shows the equivalences:

|`NumList`      | `List[Int]`|
|---------------|------------|
|`MyNil`        | `Nil`      |
|`MyCons(a, b)` | `a :: b`   |

In [ ]:
def length(list : List[Int]) : Int = {
    // BEGIN SOLUTION
    list match {
        case Nil => 0
        case firstElement :: restOfList => 1 + length(restOfList)
    }
    
    // END SOLUTION
}

In [ ]:
assert(length(List()) == 0)
assert(length(List(1, 2, 3)) == 3)

### Exercise: Higher Order functions
Define a function called `map`. `map` should takes a list of `Int`s, and a function from `Int` to `Int`, then call the function on each element and make a new list from the results.

For example, the following call:
```
map(List(1, 2, 3, 4, 5), (x) => x + 1)
```
should return in the following list
```
List(2, 3, 4, 5, 6)
```

In [ ]:
// BEGIN SOLUTION
def map(l: List[Int], f: (Int) => Int): List[Int] = l match {
    case Nil => Nil
    case e :: rest => f(e) :: map(rest, f)
}
// END SOLUTION

In [ ]:
assert(map(List(), _ + 1) == List())
assert(map(List(1), _ + 1) == List(2))
assert(map(List(1, 9, 2), _ + 1) == List(2, 10, 3))
assert(map(List(1, 2, 3, 4, 5), _ + 1) == List(2, 3, 4, 5, 6))

## Abstract Syntax Trees (AST)
We'll be reusing the AST for sets from the previous recitation (without the complement). For more on ASTs, see
https://en.wikipedia.org/wiki/Abstract_syntax_tree

$$
\begin{array}{rcl}
    \textbf{Set} & \rightarrow & Empty \\
                 &           | & Cons(\textbf{Set}, \textbf{Int}) \\
                 &           | & Intersection(\textbf{Set}, \textbf{Set}) \\
                 &           | & Union(\textbf{Set}, \textbf{Set}) \\
                 &           | & Subtraction(\textbf{Set}, \textbf{Set}) \\
\end{array}
$$

In [ ]:
sealed trait MySet
case object Empty extends MySet
case class Cons(s: MySet, n: Int) extends MySet
case class Intersection(s1: MySet, s2: MySet) extends MySet
case class Union(s1: MySet, s2: MySet) extends MySet
case class Subtraction(s1: MySet, s2: MySet) extends MySet

### Exercise: Interpreter

Scala Set API -- https://docs.scala-lang.org/overviews/collections/sets.html

Write an interpreter for the language of sets you've created. Some useful methods of Sets:

```
Set.+:           (Set[Int], Int)            => Set[Int]
Set.intersect:   (Set[Int], Set[Int])       => Set[Int]
Set.union:       (Set[Int], Set[Int])       => Set[Int]
Set.diff:        (Set[Int], Set[Int])       => Set[Int]
```

In [ ]:
def eval(set_expression: MySet): Set[Int] = {
    // BEGIN SOLUTION
    set_expression match {
        case Empty => Set()
        case Cons(s, n) => eval(s) + n
        case Intersection(s1, s2) => eval(s1) intersect eval(s2)
        case Union(s1, s2) => eval(s1) union eval(s2)
        case Subtraction(s1, s2) => eval(s1) diff eval(s2)
    }
    // END SOLUTION
}

In [ ]:
val set_1_2 = Cons(Cons(Empty, 1), 2)
val set_1_2_3 = Cons(Cons(Cons(Empty, 1), 2), 3)
val set_3 = Cons(Empty, 3)

assert(eval(Empty) == Set())
assert(eval(set_1_2) == Set(1, 2))
assert(eval(Union(set_1_2, set_3)) == Set(1, 2, 3))
assert(eval(Intersection(set_1_2, set_1_2_3)) == Set(1, 2))
assert(eval(Subtraction(set_1_2, set_1_2_3)) == Set())